In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import operator
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
import csv
import sys

In [12]:
#running on a windows machine, change filepaths as needed

train_df = pd.read_csv('Data\\train.csv')
test_df = pd.read_csv('Data\\test.csv')

In [11]:
train_df

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802


In [13]:
# Feature Names: address, dayofweek
le = preprocessing.LabelEncoder()
le.fit(np.unique(train_df.DayOfWeek.values))
train_df.DayOfWeek = le.transform(train_df.DayOfWeek)

le.fit(np.unique(train_df.Address.values))
train_df.Address = le.transform(train_df.Address)

print train_df.DayOfWeek.values

[6 6 6 ..., 1 1 1]


In [15]:
# Split this into Dev and Training Data
DEV_SIZE = 0.20

# Create boolean mask
# np.random creates a vector of random values between 0 and 1
# Those values are filtered to create a binary mask
msk = np.random.rand(len(train_df)) < DEV_SIZE

dev = train_df[msk]
dev_labels = dev.Category.values
dev = dev.drop(['Category','Dates','Descript','X','Y','Resolution','PdDistrict'], axis=1)

train = train_df[~msk]  # inverse of boolean mask
train_labels = train.Category.values
train = train.drop(['Category','Dates','Descript','X','Y','Resolution','PdDistrict'], axis=1)

print "Dev: " + str(dev.shape)
print "Train: " + str(train.shape)
print "Test: " + str(test_df.shape)

# Target Names are the categories
print train_labels

Dev: (176310, 2)
Train: (701739, 2)
Test: (884262, 7)
['WARRANTS' 'OTHER OFFENSES' 'OTHER OFFENSES' ..., 'LARCENY/THEFT'
 'VANDALISM' 'FORGERY/COUNTERFEITING']


In [16]:
model = GaussianNB()
model.fit(train,train_labels)

predicted = np.array(model.predict_proba(dev))
labels = ['Id']
for i in model.classes_:
    labels.append(i)

with open('GaussianNB.csv', 'wb') as csvfile:
    fo = csv.writer(csvfile, lineterminator='\n')
    fo.writerow(labels)

    for i, pred in enumerate(predicted):
        fo.writerow([i] + list(pred))

In [19]:
#split time into date/hour
temp = pd.DatetimeIndex(train_df['Dates'])
train_df['Date'] = temp.date
train_df['Time'] = temp.time

#remove the original dates columns
del train_df['Dates']

train_df.to_csv('Data\\train_01.csv')


In [35]:
#working with a smaller sample for ease of manipulation
#created train_350.csv which only has 350 records

train_sm = pd.read_csv('Data\\train_350.csv')
temp = pd.DatetimeIndex(train_sm['Dates'])
train_sm['Date'] = temp.date
train_sm['Time'] = temp.time
train_sm['Hour'] = temp.hour
del train_sm['Dates']

# Feature Names: address, dayofweek
le = preprocessing.LabelEncoder()
le.fit(np.unique(train_sm.DayOfWeek.values))
train_sm.DayOfWeek = le.transform(train_sm.DayOfWeek)

le.fit(np.unique(train_sm.Address.values))
train_sm.Address = le.transform(train_sm.Address)

print train_sm

            Category                                           Descript  \
0           WARRANTS                                     WARRANT ARREST   
1     OTHER OFFENSES                           TRAFFIC VIOLATION ARREST   
2     OTHER OFFENSES                           TRAFFIC VIOLATION ARREST   
3      LARCENY/THEFT                       GRAND THEFT FROM LOCKED AUTO   
4      LARCENY/THEFT                       GRAND THEFT FROM LOCKED AUTO   
5      LARCENY/THEFT                     GRAND THEFT FROM UNLOCKED AUTO   
6      VEHICLE THEFT                                  STOLEN AUTOMOBILE   
7      VEHICLE THEFT                                  STOLEN AUTOMOBILE   
8      LARCENY/THEFT                       GRAND THEFT FROM LOCKED AUTO   
9      LARCENY/THEFT                       GRAND THEFT FROM LOCKED AUTO   
10     LARCENY/THEFT                       PETTY THEFT FROM LOCKED AUTO   
11    OTHER OFFENSES                        MISCELLANEOUS INVESTIGATION   
12         VANDALISM     

In [36]:
dummy_cat = pd.get_dummies(train_sm['Category'], prefix='c')
dummy_dist = pd.get_dummies(train_sm['PdDistrict'], prefix='d')
print dummy_dist

     d_BAYVIEW  d_CENTRAL  d_INGLESIDE  d_MISSION  d_NORTHERN  d_PARK  \
0            0          0            0          0           1       0   
1            0          0            0          0           1       0   
2            0          0            0          0           1       0   
3            0          0            0          0           1       0   
4            0          0            0          0           0       1   
5            0          0            1          0           0       0   
6            0          0            1          0           0       0   
7            1          0            0          0           0       0   
8            0          0            0          0           0       0   
9            0          1            0          0           0       0   
10           0          1            0          0           0       0   
11           0          0            0          0           0       0   
12           0          0            0          0  

In [37]:
train_smd = pd.concat([train_sm, dummy_cat, dummy_dist], axis=1)
del train_smd['Category']
del train_smd['Descript']
del train_smd['Resolution']
del train_smd['PdDistrict']
print train_smd

     DayOfWeek  Address           X          Y        Date      Time  Hour  \
0            0      248 -122.425892  37.774599  2015-05-13  23:53:00    23   
1            0      248 -122.425892  37.774599  2015-05-13  23:53:00    23   
2            0      265 -122.424363  37.800414  2015-05-13  23:33:00    23   
3            0       67 -122.426995  37.800873  2015-05-13  23:30:00    23   
4            0       28 -122.438738  37.771541  2015-05-13  23:30:00    23   
5            0       22 -122.403252  37.713431  2015-05-13  23:30:00    23   
6            0      199 -122.423327  37.725138  2015-05-13  23:30:00    23   
7            0      238 -122.371274  37.727564  2015-05-13  23:30:00    23   
8            0      168 -122.508194  37.776601  2015-05-13  23:00:00    23   
9            0      234 -122.419088  37.807802  2015-05-13  23:00:00    23   
10           0      234 -122.419088  37.807802  2015-05-13  22:58:00    22   
11           0        9 -122.487983  37.737667  2015-05-13  22:3